**Please do not run the notebook as this will rewrite the output of the models.**
**PS. The graphs are interactive. Thank you!**

In [ ]:
#install essentials
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import altair as alt
from altair import datum
import datetime
from sklearn import linear_model
import sklearn
import pickle
#import seaborn as sns

## Data Cleaning Checklist
- Drop null values or replace with mean
- Normalise and scale (IF MODEL REQUIRES IT)
- Get dates right
- Pretty much it

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
path = "/content/drive/MyDrive/mstw/Brent Oil Futures Historical Data (1).csv"
path2 = "/content/drive/MyDrive/mstw/gscpi_data.xls - GSCPI Monthly Data.csv"
path3 = "/content/drive/MyDrive/mstw/owid-covid-data.csv"
path4 = "/content/drive/MyDrive/mstw/chinaexports.csv"
path5 = "/content/drive/MyDrive/mstw/fbx-global-container-freight-index.csv"
oil_prices = pd.read_csv(path)
gscpi = pd.read_csv(path2)
cases = pd.read_csv(path3)
exports = pd.read_csv(path4,thousands=',')
fbx = pd.read_csv(path5)

# Data Cleaning(Oil Prices)



In [ ]:
oil_prices

,Date,Price,Open,High,Low,Vol.,Change %
0,"Aug 19, 2022",95.78,96.80,97.84,94.26,NaN,-0.84%
1,"Aug 18, 2022",96.59,93.29,97.42,93.05,208.55K,3.14%
2,"Aug 17, 2022",93.65,92.67,94.48,91.51,199.19K,1.42%
3,"Aug 16, 2022",92.34,94.01,95.95,91.71,217.75K,-2.90%
4,"Aug 15, 2022",95.10,97.72,98.17,92.78,217.57K,-3.11%
...,...,...,...,...,...,...,...
935,"Jan 08, 2019",58.72,57.64,58.87,57.13,288.25K,2.42%
936,"Jan 07, 2019",57.33,57.41,58.93,57.26,296.58K,0.47%
937,"Jan 04, 2019",57.06,55.67,58.31,55.34,321.11K,1.98%
938,"Jan 03, 2019",55.95,54.29,56.30,53.93,278.36K,1.89%


In [ ]:
#@title Checking for Null Values
oil_prices['Price'].isnull().values.any()

False

In [ ]:
#@title Converting to datetime format
oil_prices["Date"] = pd.to_datetime(oil_prices["Date"])

In [ ]:
#@title Saving Cleaned data
oil_prices.to_csv('oil_prices_cleaned.csv')

# Data Cleaning (GSCPI)
GSCPI: Global Supply Chain Pressure Index


In [ ]:
#@title Clearing Null Values
gscpi[['Date','GSCPI']].isnull().values.sum()
gscpi = gscpi[['Date', 'GSCPI']][~gscpi['GSCPI'].isnull()]

In [ ]:
#@title Changing Date-time format
gscpi["Date"] = pd.to_datetime(gscpi["Date"])

In [ ]:
#@title Saving Cleaned data
gscpi.to_csv('gscpi_cleaned.csv')

# Data Cleaning (Covid Cases)


In [ ]:
#@title Filtering China Cases
cases = cases[cases['location'] == 'China']
cases.rename(columns = {'date':'Date'}, inplace = True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
#@title Clearing Null Values
cases['new_cases_smoothed'].isnull().values.sum()
cases = cases[~cases['new_cases_smoothed'].isnull()]
cases = cases[['Date','new_cases_smoothed']]

In [ ]:
#@title Changing Date-time format
cases["Date"] = pd.to_datetime(cases["Date"])
cases

,Date,new_cases_smoothed
38715,2020-01-28,706.714
38716,2020-01-29,789.000
38717,2020-01-30,1069.286
38718,2020-01-31,1266.714
38719,2020-02-01,1496.000
...,...,...
39645,2022-08-15,1747.143
39646,2022-08-16,2322.857
39647,2022-08-17,2463.286
39648,2022-08-18,2861.286


In [ ]:
#@title Saving Cleaned data
cases.to_csv('cases_cleaned.csv')

# Data Cleaning (Exports from China)

In [ ]:
#@title Changing Date-time format
exports['Date'] = exports['Date'].astype(str)
exports["Date"] = pd.to_datetime(exports["Date"], format = "%Y.%m")
exports.head()

,Date,Total,Export,Import,Balance
0,2019-01-01,397594,218003,179591,38412
1,2019-02-01,267608,135324,132284,3039
2,2019-03-01,365000,198232,166769,31463
3,2019-04-01,374020,193551,180469,13082
4,2019-05-01,386579,213920,172659,41260


In [ ]:
#@title Saving Cleaned data
exports.to_csv('exports_cleaned.csv')

# Data Cleaning (FBX)

In [ ]:
#@title Changing Date-time format
fbx['Date'] = fbx['Date'].astype(str)
fbx["Date"] = pd.to_datetime(fbx["Date"])
fbx

,Date,Index Price (USD)
0,2019-04-01,1515
1,2019-11-01,1559
2,2019-01-18,1575
3,2019-01-25,1603
4,2019-01-02,1588
...,...,...
185,2022-07-22,6319
186,2022-07-29,6120
187,2022-05-08,6139
188,2022-12-08,5956


In [ ]:
#@title Saving Cleaned data
fbx.to_csv('fbx_cleaned.csv')

# Data Analysis


In [ ]:
alt.data_transformers.disable_max_rows()
chart = alt.Chart(oil_prices[oil_prices['Date'] >= '2020-01-01']).mark_line(tooltip=True).encode(
  alt.X('Date',axis=alt.Axis(format = "%B %Y", labelAngle=-45)),
  alt.Y('Price'),
  color = alt.value('green'),
  tooltip=['Date','Price']
).properties(
    title='Price of Brent Oil Futures'
)

chart1 = alt.Chart(gscpi[gscpi['Date'] >= '2020-01-01']).mark_line(tooltip=True).encode(
  alt.X('Date',axis=alt.Axis(format = "%B %Y", labelAngle=-45)),
  alt.Y('GSCPI'),
  tooltip=['Date','GSCPI']
).properties(
    title='GSCPI'
)

chart2 = alt.Chart(cases).mark_line(tooltip=True).encode(
  alt.X('Date',axis=alt.Axis(format = "%B %Y", labelAngle=-45)),
  alt.Y('new_cases_smoothed:Q', title = 'Daily New Covid Cases (log)', scale=alt.Scale(type="log")),
  
  tooltip=['Date','new_cases_smoothed'],
).properties(
    title='Log of Daily New Covid Cases'
)

rules = alt.Chart(pd.DataFrame({
  'Date': ['2020-02-20','2020-02-24','2020-04-08'],
  'color': ['orange','red','blue'],
  'Event':['Hubei Lockdown Commenced','Russia launched a full-scale military invasion into Ukraine', 'Wuhan Lockdown Ended']
})).mark_rule().encode(
  x='Date:T',
  color=alt.Color('color:N', scale=None),
  tooltip = ['Event']
)

chart = chart + rules 
chart1 = chart1 + rules
chart2 = chart2 + rules

# Saving charts
chart.save('oil_prices_chart.html')
chart1.save('gscpi_chart.html')
chart2.save('covid_chart.html')
combined = chart1|chart2|chart
combined.save('combined.html')
combined

alt.HConcatChart(...)

In [ ]:
alt.data_transformers.disable_max_rows()
chart3 = alt.Chart(exports[exports['Date'] >= '2019-01-01']).mark_line(tooltip=True).encode(
  alt.X('Date',axis=alt.Axis(format = "%B %Y", labelAngle=-45)),
  alt.Y('Export'),
  color = alt.value('brown'),
  tooltip=['Date','Export']
).properties(
    title='Exports from China (million USD)'
)
chart3.save('exports_china.html')

chart4 = alt.Chart(fbx[fbx['Date'] >= '2019-01-01']).mark_line(tooltip=True).encode(
  alt.X('Date',axis=alt.Axis(format = "%B %Y", labelAngle=-45)),
  alt.Y('Index Price (USD)'),
  color = alt.value('green'),
  tooltip=['Date','Index Price (USD)']
).properties(
    title='FBX Index'
)


rules = alt.Chart(pd.DataFrame({
  'Date': ['2020-02-20','2020-02-24','2020-04-08'],
  'color': ['orange','red','blue'],
  'Event':['Hubei Lockdown Commenced','Russia launched a full-scale military invasion into Ukraine', 'Wuhan Lockdown Ended']
})).mark_rule().encode(
  x='Date:T',
  color=alt.Color('color:N', scale=None),
  tooltip = ['Event']
)

chart3 = chart3 + rules
chart4 = chart4 + rules

# Saving Charts
chart3.save('exports.html')
chart4.save('fbx.html')
combined2 = chart3|chart4
combined.save('combined.html')
combined2

alt.HConcatChart(...)

In [ ]:
merged = pd.merge(oil_prices[oil_prices['Date'] >= '2020-01-01'], cases, on='Date',how='inner')
merged.head(5)

,Date,Price,Open,High,Low,Vol.,Change %,new_cases_smoothed
0,2022-08-19,95.78,96.80,97.84,94.26,NaN,-0.84%,3033.571
1,2022-08-18,96.59,93.29,97.42,93.05,208.55K,3.14%,2861.286
2,2022-08-17,93.65,92.67,94.48,91.51,199.19K,1.42%,2463.286
3,2022-08-16,92.34,94.01,95.95,91.71,217.75K,-2.90%,2322.857
4,2022-08-15,95.10,97.72,98.17,92.78,217.57K,-3.11%,1747.143


In [ ]:
merged['Month'] = merged['Date'].dt.to_period('M')
grouped_month = merged.groupby(merged['Month']).mean()

In [ ]:
gscpi['Month'] = gscpi['Date'].dt.to_period('M')
exports['Month'] = exports['Date'].dt.to_period('M')
merged2 = pd.merge(grouped_month, gscpi[gscpi['Date'] >= '2020-01-01'], on='Month', how='inner')

In [ ]:
merged2['cases_log'] = np.log(merged2['new_cases_smoothed'])
alt.Chart(merged2[['Price','GSCPI','cases_log']]).mark_rect().encode(
    alt.X('GSCPI:O', bin=alt.Bin(step=0.2)),
    alt.Y('cases_log:O',scale=alt.Scale(type="log"), bin=alt.Bin(step=1), title='Daily New COVID Cases'),
    color='Price:Q'
).properties(
    title='Heat Map of Daily New COVID Cases against GSCPI'
)

alt.Chart(...)

In [ ]:
fbx_price = pd.merge(fbx, oil_prices, on='Date', how='inner')
chart_oil_fbx = alt.Chart(fbx_price).mark_point().encode(
    alt.X('Index Price (USD)',title='FBX Price (USD)'),
    alt.Y('Price'),
    tooltip = ['Index Price (USD)', 'Price']
).properties(
    title = 'Crude Oil Prices against FBX'
)
chart_oil_fbx

alt.Chart(...)

**Price to GSCPI correlation**

In [ ]:
oil_prices_copy = oil_prices.copy()
oil_prices_copy['Month'] = oil_prices_copy['Date'].dt.to_period('M')
old_prices_grouped = oil_prices_copy.groupby(oil_prices_copy['Month']).mean()
# old_prices_grouped
# gscpi
price_to_gscpi = pd.merge(old_prices_grouped, gscpi, on='Month',how='inner')
price_to_gscpi['Price'].corr(price_to_gscpi['GSCPI'])

0.29009726943865394

In [ ]:
price_to_cases = pd.merge(oil_prices, cases, on='Date',how='inner')
price_to_cases['Price'].corr(price_to_cases['new_cases_smoothed'])

0.3403433492370552

In [ ]:
price_to_fbx = pd.merge(oil_prices, fbx, on='Date',how='inner')
price_to_fbx['Price'].corr(price_to_fbx['Index Price (USD)'])

0.6794279624183556

In [ ]:
price_to_exports = pd.merge(old_prices_grouped, exports, on='Month',how='inner')
price_to_exports['Price'].corr(price_to_exports['Export'])

0.5169781892373911

In [ ]:
price_to_exports['Price'].corr(price_to_exports['Balance'])

0.29605406999016604

In [ ]:
fbx = fbx.sort_values(by=['Date']) 

In [ ]:
fbx_copy = fbx.copy()
fbx_copy['Month'] = fbx_copy['Date'].dt.to_period('M')
fbx_copy['Month'] = fbx_copy['Month'].astype(str)
fbx_copy

chart100 = alt.Chart(fbx_copy).mark_boxplot(extent='min-max').encode(
  alt.X('Month',axis=alt.Axis(labelAngle=-45)),
  alt.Y('Index Price (USD)', title='FBX Price (USD)'),
  color = alt.value('green')
).properties(
    title='Box Plot of FBX Price (USD)'
)
chart100

alt.Chart(...)

# Correlation Matrix and Linear Regression Model to Predict Oil Futures Prices


In [ ]:

exports2=exports.copy()
exports2["Month"] = exports2["Date"].apply(lambda x: x.strftime('%Y-%m'))
exports2['Month'] = exports2['Date'].dt.to_period('M')


merged3 = pd.merge(oil_prices[oil_prices['Date'] >= '2019-01-01'], cases, on='Date',how='inner')
merged3 = pd.merge(fbx[fbx['Date'] >= '2019-01-01'], merged3, on='Date',how='inner')
merged3['Month'] = merged3['Date'].dt.to_period('M')
grouped_month2 = merged3.groupby(merged3['Month']).mean() #EXPERIMENT: ORIGINALLY USED .MEAN()
grouped_month2 = pd.merge(grouped_month2, exports2[exports2['Date'] >= '2019-01-01'], on='Month', how='inner')
grouped_month2 = pd.merge(grouped_month2, gscpi[gscpi['Date'] >= '2019-01-01'], on='Month', how='inner')


global_df=grouped_month2[["Month", "Index Price (USD)", "new_cases_smoothed", "Export",  "Import",  "Price", "Balance", "GSCPI"]]
global_df=global_df.rename(columns = {"Index Price (USD)": "FBX", "Price": "Oil Futures Price", "new_cases_smoothed":"New COVID Cases (China)"})


cor_data = (global_df.corr().stack().reset_index().rename(columns={0: 'correlation', 'level_0': 'variable', 'level_1': 'variable2'}))
cor_data['correlation_label'] = cor_data['correlation'].map('{:.2f}'.format)  # Round to 2 decimal

base = alt.Chart(cor_data).encode(
      alt.X('variable2:O',axis=alt.Axis(labelAngle=-45)),
    y='variable:O'    
).properties(
    title='Correlation Matrix of China Exports, Imports, Balances, Oil Futures Prices, FBX, GSCPI and COVID Cases in China'
)

# Text layer with correlation labels
# Colors are for easier readability
text = base.mark_text().encode(
    text='correlation_label',
    color=alt.condition(
        alt.datum.correlation > 0.5, 
        alt.value('white'),
        alt.value('black')
    )
)

# The correlation heatmap itself
cor_plot = base.mark_rect().encode(
    color='correlation:Q', 
).properties(
    width=300,
    height=300
)

cor_plot + text # The '+' means overlaying the text and rect layer

alt.LayerChart(...)

In [ ]:
ml_df = global_df[['FBX', 'Export', "Import", "Oil Futures Price"]]
predict = "Oil Futures Price"

X = np.array(ml_df.drop([predict], axis=1))
y = np.array(ml_df[predict])

In [ ]:
# TRANING PROCESS
best = 0
for _ in range(10000):
    x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size=0.1)

    # Using Linear Regression model
    linear = linear_model.LinearRegression()
    linear.fit(x_train, y_train)
    acc = linear.score(x_test, y_test)

    if acc > best:
        with open("predict_oil_futures_price_v2.pickle", "wb") as file:
            pickle.dump(linear, file)
        best = acc
    print(acc)
print("Best: ", best)

In [ ]:
# merged4 = pd.merge(oil_prices[oil_prices['Date'] >= '2019-01-01'], cases, on='Date',how='inner')
merged4 = pd.merge(fbx, oil_prices, on='Date',how='inner')
merged4['Month'] = merged4['Date'].dt.to_period('M')
grouped_months4 = merged4.groupby(merged4['Month']).mean() #EXPERIMENT: ORIGINALLY USED .MEAN()
grouped_months4 = pd.merge(grouped_months4, exports2[exports2['Date'] >= '2019-01-01'], on='Month', how='inner')
grouped_months4 = pd.merge(grouped_months4, gscpi[gscpi['Date'] >= '2019-01-01'], on='Month', how='inner')


global_df_2=grouped_months4[["Month", "Index Price (USD)", "Export",  "Import",  "Price", "Balance", "GSCPI"]]
global_df_2=global_df_2.rename(columns = {"Index Price (USD)": "FBX", "Price": "Oil Futures Price"})


cor_data2 = (global_df_2.corr().stack().reset_index().rename(columns={0: 'correlation', 'level_0': 'variable', 'level_1': 'variable2'}))
cor_data2['correlation_label'] = cor_data2['correlation'].map('{:.2f}'.format)  # Round to 2 decimal

base = alt.Chart(cor_data2).encode(
      alt.X('variable2:O',axis=alt.Axis(labelAngle=-45)),
    y='variable:O'    
).properties(
    title='Correlation Matrix of China Exports, Imports, Balances, Oil Futures Prices, FBX and GSCPI'
)

# Text layer with correlation labels
# Colors are for easier readability
text = base.mark_text().encode(
    text='correlation_label',
    color=alt.condition(
        alt.datum.correlation > 0.5, 
        alt.value('white'),
        alt.value('black')
    )
)

# The correlation heatmap itself
cor_plot2 = base.mark_rect().encode(
    color='correlation:Q', 
).properties(
    width=300,
    height=300
)

cor_plot2 + text # The '+' means overlaying the text and rect layer

alt.LayerChart(...)

In [ ]:
ml_df_2 = global_df_2[['Export', 'FBX', "Oil Futures Price"]]

predict = "Oil Futures Price"

X = np.array(ml_df_2.drop([predict], axis=1))
y = np.array(ml_df_2[predict])

In [ ]:
# TRANING PROCESS
best = 0
for _ in range(30000):
    x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size=0.1)

    # Using Linear Regression model
    linear_2 = linear_model.Ridge()
    linear_2.fit(x_train, y_train)
    acc = linear_2.score(x_test, y_test)

    if acc > best:
        with open("predict_oil_futures_price_improved_v3.pickle", "wb") as file:
            pickle.dump(linear_2, file)
        best = acc
print("Best: ", best)

In [ ]:
ml_df_3 = global_df_2[['Export', 'FBX', 'GSCPI', 'Import', "Oil Futures Price"]]

predict = "Oil Futures Price"

X = np.array(ml_df_3.drop([predict], axis=1))
y = np.array(ml_df_3[predict])

In [ ]:
# TRANING PROCESS
best = 0
for _ in range(10000):
    x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size=0.1)

    # Using Linear Regression model
    linear_2 = linear_model.Ridge()
    linear_2.fit(x_train, y_train)
    acc = linear_2.score(x_test, y_test)

    if acc > best:
        with open("predict_oil_futures_price_more_improved_v1.pickle", "wb") as file:
            pickle.dump(linear_2, file)
        best = acc
print("Best: ", best)

In [ ]:
# Create predict oil futures price function
def predict_oil_futures_price(exp, fbx, gscpi, imp):
  pickle_in = open("/content/drive/MyDrive/mstw/predict_oil_futures_price_more_improved_v1.pickle", "rb")
  linear_predict = pickle.load(pickle_in)

  print(linear_predict.predict([[exp, fbx, gscpi, imp]]))

In [ ]:
predict_oil_futures_price(305256, 9767.000000, 2.68, 237203)

[100.34274735]


# Algotrading

In [ ]:
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense
from tensorflow.keras.optimizers import Adam
import random
from collections import deque
import math

class Agent:
	def __init__(self, state_size, is_eval=False, model_name="algotrader"):
		self.state_size = state_size
		self.action_size = 3 # buy, sell, or hold
		self.memory = deque(maxlen=1000)
		self.inventory = []
		self.model_name = model_name
		self.is_eval = is_eval

		self.gamma = 0.95
		self.epsilon = 1.0
		self.epsilon_min = 0.01
		self.epsilon_decay = 0.995

		self.model = load_model("models/" + model_name) if is_eval else self._model()

	def _model(self):
		model = Sequential()
		model.add(Dense(units=32, input_dim=self.state_size, activation="relu"))#basic layers and stuff
		model.add(Dense(units=64, activation="relu")) #use relu bc of mean squared error
		model.add(Dense(units=128, activation="relu"))
		model.add(Dense(self.action_size, activation="linear"))
		model.compile(loss="mse", optimizer=Adam(lr=0.001))

		return model

	def act(self, state):
		if not self.is_eval and random.random() <= self.epsilon:
			return random.randrange(self.action_size)

		options = self.model.predict(state)
		return np.argmax(options[0])

	def expReplay(self, batch_size):
		temp_batch = []
		l = len(self.memory)
		for i in range(l - batch_size + 1, l):
			temp_batch.append(self.memory[i])

		for state, action, reward, next_state, done in temp_batch:
			target = reward
			if not done:
				target = reward + self.gamma * np.amax(self.model.predict(next_state)[0])

			target_f = self.model.predict(state)
			target_f[0][action] = target
			self.model.fit(state, target_f, epochs=1, verbose=0)

		if self.epsilon > self.epsilon_min:
			self.epsilon *= self.epsilon_decay 

# format price function
def formatPrice(n):
	return ("-$" if n < 0 else "$") + "{0:.2f}".format(abs(n))

# returns the sigmoid
def sigmoid(x):
	return 1 / (1 + math.exp(-x))

# returns an an n-day state ending at time t
def getState(data, t, n):
	d = t - n + 1
	block = data[d:t + 1] if d >= 0 else -d * [data[0]] + data[0:t + 1] # pad with t0
	res = []
	for i in range(n - 1):
		res.append(sigmoid(block[i + 1][0] - block[i][0]))

	return np.array([res])
 


window_size, episode_count =  10,5
buyCount = 0
sellCount = 0


agent = Agent(window_size)
data = global_df_2[["Oil Futures Price", "Export", "Import", "GSCPI", "FBX" ]].values.tolist()
l = len(data) - 1
batch_size = 15

for e in range(episode_count + 1):
	print("Episode " + str(e) + "/" + str(episode_count))
	state = getState(data, 0, window_size + 1)

	total_profit = 0
	agent.inventory = []

	for t in range(l):
		action = agent.act(state)

		# hold
		next_state = getState(data, t + 1, window_size + 1)
		reward = 0 #no reward

		if action == 1: # buy
			buyCount +=1
			agent.inventory.append(data[t][0])
			print("Buy: " + formatPrice(data[t][0]))

		elif action == 2 and len(agent.inventory) > 0: # sell
			bought_price = agent.inventory.pop(0)
			reward = max(data[t][0] - bought_price, 0)
			total_profit += data[t][0] - bought_price
			print("Sell: " + formatPrice(data[t][0]) + " | Profit: " + formatPrice(data[t][0] - bought_price))
			sellCount +=1

		done = True if t == l - 1 else False
		agent.memory.append((state, action, reward, next_state, done))
		state = next_state

		if done:
			print("--------------------------------")
			print("Total Profit: " + formatPrice(total_profit))
			print("Barrels Bought:  " + str(buyCount))
			print("Barrels Sold:  " + str(sellCount))
			print("Current Barrel Price:  " + str(data[t][0]))
			print("Estimated Return:  " + str((total_profit/buyCount/70*100) ))
			print(agent.inventory)
			print("--------------------------------")

		if len(agent.memory) > batch_size:
			agent.expReplay(batch_size)

	if e % 10 == 0:
		agent.model.save("../working/model_ep" + str(e))

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Episode 0/5
Buy: $65.16
Sell: $70.96 | Profit: $5.81
Buy: $68.82
Sell: $63.43 | Profit: -$5.38
Buy: $65.63
Sell: $61.23 | Profit: -$4.40
Buy: $34.22
Buy: $27.00
Buy: $33.17
Buy: $43.24
Sell: $44.84 | Profit: $10.62
Buy: $41.71
Sell: $44.37 | Profit: $17.37
Sell: $52.26 | Profit: $19.09
Sell: $62.63 | Profit: $19.38
Buy: $65.72
Sell: $68.18 | Profit: $26.47
Buy: $75.30
Sell: $69.78 | Profit: $4.06
Sell: $75.04 | Profit: -$0.25
Buy: $75.81
Sell: $85.15 | Profit: $9.34
Buy: $108.00
--------------------------------
Total Profit: $102.10
Barrels Bought:  12
Barrels Sold:  11
Current Barrel Price:  113.12
Estimated Return:  12.154900793650791
[107.995]
--------------------------------
Episode 1/5
Buy: $59.14
Sell: $65.16 | Profit: $6.02
Buy: $63.43
Buy: $60.08
Buy: $62.14
Sell: $62.70 | Profit: -$0.73
Buy: $65.63
Sell: $61.23 | Profit: $1.16
Sell: $54.90 | Profit: -$7.23
Buy: $34.22
Buy: $27.00
Buy: $43.24
Sell: $41.71 | Profit: -$23.92
Sell: $44.37 | Profit: $10.15
Sell: $52.26 | Profit: $2

In [ ]:
global_df_2[["Oil Futures Price", "Export", "Import", "GSCPI", "FBX"]].values.tolist()

[[59.136, 218003.0, 179591.0, 0.54, 1498.0],
 [59.136, 213025.0, 170478.0, 0.54, 1498.0],
 [65.15666666666667, 135324.0, 132284.0, 0.13, 1479.0],
 [67.11, 198232.0, 166769.0, 0.2, 1331.25],
 [70.96333333333334, 193551.0, 180469.0, 0.03, 1372.6666666666667],
 [68.8175, 213920.0, 172659.0, -0.65, 1333.0],
 [63.432500000000005, 212406.0, 162816.0, -0.48, 1319.75],
 [62.965, 221784.0, 177700.0, -0.46, 1358.0],
 [60.075, 214852.0, 180243.0, -0.32, 1394.75],
 [62.13666666666666, 218196.0, 179179.0, 0.13, 1313.6666666666667],
 [62.7025, 221470.0, 183852.0, 0.12, 1383.5],
 [65.63, 238268.0, 191057.0, 0.01, 1411.25],
 [61.23333333333333, 211588.0, 156921.0, 0.05, 1548.0],
 [61.23333333333333, 236230.0, 178908.0, 0.05, 1548.0],
 [54.9025, 80379.0, 142372.0, 1.13, 1593.5],
 [34.222500000000004, 184572.0, 164603.0, 2.51, 1388.5],
 [27.0, 199483.0, 154547.0, 3.19, 1646.0],
 [33.17, 206428.0, 144811.0, 2.65, 1493.25],
 [40.3875, 212847.0, 168167.0, 2.4, 1793.75],
 [43.242000000000004, 236839.0, 1765